In [1]:
import re 
import csv
import json
import nltk
import sklearn
import pandas as pd
import numpy as np 
from pprint import pprint
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt
from reviewsData import * 
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
pd.set_option("display.max_colwidth",-1)

train data size: (34092, 3) ..

test data size: (14613, 3) ..

Unlabeled train data size : (60797, 2) ..



In [2]:
tf_vectorizer = TfidfVectorizer(min_df=5, 
                                    max_df = 0.8,
                                    sublinear_tf=True,
                                    use_idf=True,
                                    analyzer='word',
                                    ngram_range=(1,2),
                                    stop_words = None)



train_vectors = tf_vectorizer.fit_transform(train['reviews'])
test_vectors = tf_vectorizer.transform(test['reviews'])


In [39]:
tf_stop = TfidfVectorizer(min_df=5, 
                                    max_df = 0.8,
                                    sublinear_tf=True,
                                    use_idf=True,
                                    analyzer='word',
                                    ngram_range=(1,2),
                                    stop_words = 'english')

train_vectors_stop = tf_stop.fit_transform(train['reviews'])
test_vectors_stop = tf_stop.transform(test['reviews'])

In [45]:
# Trigram
tf = TfidfVectorizer(min_df=3, 
                                    max_df = 0.8,
                                    sublinear_tf=True,
                                    use_idf=True,
                                    analyzer='word',
                                    ngram_range=(1,3),
                                    stop_words = None)

train_vectors_tri = tf.fit_transform(train['reviews'])
test_vectors_tri = tf.transform(test['reviews'])

In [46]:
# trigram
from sklearn.linear_model import SGDClassifier as SGD
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import make_scorer
#f1_scorer = make_scorer(f1_score, pos_label="postive",average='binary')
precision_scorer = make_scorer(precision_score, pos_label="postive")

sgd_params = {'alpha': [0.00006, 0.00007, 0.00008, 0.0001, 0.0005,0.0003]} # Regularization parameter
    
model_SGD = GridSearchCV(SGD(random_state = 0, shuffle = True, penalty='l2',
                         loss = 'hinge'), 
                         sgd_params, scoring = precision_scorer, cv = 30) # Find out which regularization parameter works the best. 
                            
model_SGD.fit(train_vectors_tri, train['sentiment']) # Fit the model.
SGD_result = model_SGD.predict(test_vectors_tri)

target_names = ['postive', 'negative']
print('Accuracy score = ',accuracy_score(test['sentiment'], SGD_result ),'..\n')
print(classification_report(test['sentiment'], SGD_result , target_names=target_names))

Accuracy score =  0.84089509341 ..

             precision    recall  f1-score   support

    postive       0.86      0.78      0.82      6862
   negative       0.82      0.89      0.86      7751

avg / total       0.84      0.84      0.84     14613



In [40]:
# stopword remove
from sklearn.linear_model import SGDClassifier as SGD
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import make_scorer
#f1_scorer = make_scorer(f1_score, pos_label="postive",average='binary')
precision_scorer = make_scorer(precision_score, pos_label="postive")

sgd_params = {'alpha': [0.00006, 0.00007, 0.00008, 0.0001, 0.0005,0.0003]} # Regularization parameter
    
model_SGD = GridSearchCV(SGD(random_state = 0, shuffle = True, penalty='l2',
                         loss = 'hinge'), 
                         sgd_params, scoring = precision_scorer, cv = 30) # Find out which regularization parameter works the best. 
                            
model_SGD.fit(train_vectors_stop, train['sentiment']) # Fit the model.
SGD_result = model_SGD.predict(test_vectors_stop)

target_names = ['postive', 'negative']
print('Accuracy score = ',accuracy_score(test['sentiment'], SGD_result ),'..\n')
print(classification_report(test['sentiment'], SGD_result , target_names=target_names))


Accuracy score =  0.831725176213 ..

             precision    recall  f1-score   support

    postive       0.86      0.77      0.81      6862
   negative       0.81      0.89      0.85      7751

avg / total       0.83      0.83      0.83     14613



In [36]:
from sklearn.linear_model import SGDClassifier as SGD
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import make_scorer
#f1_scorer = make_scorer(f1_score, pos_label="postive",average='binary')
precision_scorer = make_scorer(precision_score, pos_label="postive")

sgd_params = {'alpha': [0.00006, 0.00007, 0.00008, 0.0001, 0.0005,0.0003]} # Regularization parameter
    
model_SGD = GridSearchCV(SGD(random_state = 0, shuffle = True, penalty='l2',
                         loss = 'hinge'), 
                         sgd_params, scoring = precision_scorer, cv = 30) # Find out which regularization parameter works the best. 
                            
model_SGD.fit(train_vectors, train['sentiment']) # Fit the model.
SGD_result = model_SGD.predict(test_vectors)


In [37]:
# SVM + penalty l1

target_names = ['postive', 'negative']
print('Accuracy score = ',accuracy_score(test['sentiment'], SGD_result ),'..\n')
print(classification_report(test['sentiment'], SGD_result , target_names=target_names))

Accuracy score =  0.826524327653 ..

             precision    recall  f1-score   support

    postive       0.85      0.77      0.81      6862
   negative       0.81      0.87      0.84      7751

avg / total       0.83      0.83      0.83     14613



In [35]:
# SVM + penalty l2

target_names = ['postive', 'negative']
print('Accuracy score = ',accuracy_score(test['sentiment'], SGD_result ),'..\n')
print(classification_report(test['sentiment'], SGD_result , target_names=target_names))

Accuracy score =  0.842537466639 ..

             precision    recall  f1-score   support

    postive       0.86      0.79      0.82      6862
   negative       0.83      0.89      0.86      7751

avg / total       0.84      0.84      0.84     14613



In [27]:
# SVM no specifique penatly (defaut l2)

target_names = ['postive', 'negative']
print('Accuracy score = ',accuracy_score(test['sentiment'], SGD_result ),'..\n')
print(classification_report(test['sentiment'], SGD_result , target_names=target_names))

Accuracy score =  0.842537466639 ..

             precision    recall  f1-score   support

    postive       0.86      0.79      0.82      6862
   negative       0.83      0.89      0.86      7751

avg / total       0.84      0.84      0.84     14613



In [25]:
# logistic Regression log penalty

target_names = ['postive', 'negative']
print('Accuracy score = ',accuracy_score(test['sentiment'], SGD_result ),'..\n')
print(classification_report(test['sentiment'], SGD_result , target_names=target_names))

Accuracy score =  0.837131321426 ..

             precision    recall  f1-score   support

    postive       0.85      0.79      0.82      6862
   negative       0.83      0.88      0.85      7751

avg / total       0.84      0.84      0.84     14613



In [22]:
target_names = ['postive', 'negative']
print('Accuracy score = ',accuracy_score(test['sentiment'], SGD_result ),'..\n')
print(classification_report(test['sentiment'], SGD_result , target_names=target_names))

Accuracy score =  0.839184287963 ..

             precision    recall  f1-score   support

    postive       0.85      0.80      0.82      6862
   negative       0.83      0.88      0.85      7751

avg / total       0.84      0.84      0.84     14613



In [15]:
target_names = ['postive', 'negative']
print('Accuracy score = ',accuracy_score(test['sentiment'], SGD_result ),'..\n')
print(classification_report(test['sentiment'], SGD_result , target_names=target_names))



Accuracy score =  0.839115855745 ..

             precision    recall  f1-score   support

    postive       0.85      0.79      0.82      6862
   negative       0.83      0.88      0.85      7751

avg / total       0.84      0.84      0.84     14613



In [33]:
from sklearn.naive_bayes import MultinomialNB as MNB

precision_scorer = make_scorer(f1_score, pos_label="postive")
model_NB = MNB()
model_NB.fit(train_vectors, train['sentiment'])
SGD_result_NB = model_NB.predict(test_vectors)
from sklearn.cross_validation import cross_val_score
import numpy as np

np.mean(cross_val_score(model_NB,test_vectors,test['sentiment'],cv=20, scoring=precision_scorer))


0.82686741684050813